In [ ]:
#Assignment 8

In [ ]:
!pip install rasterstats

In [ ]:
import os
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
from shapely.ops import transform
import matplotlib.pyplot as plt
import pandas as pd
import pyproj
import numpy as np
import json
import folium as fm

In [ ]:
# Load your shapefile
shapefile_path = '../../_data/INEI_LIMITE_DEPARTAMENTAL/INEI_LIMITE_DEPARTAMENTAL.shp'
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs('esri:54009')

In [ ]:
### Obtener datos de los rasters

In [ ]:
# # Loop

# # Definir los archivos raster y los nombres de los DataFrames
# rasters = {
#         'df10_10':'../../_data/rasters_peru/R10_C10.tif',
#         'df10_11':'../../_data/rasters_peru/R10_C11.tif',
#         'df10_12':'../../_data/rasters_peru/R10_C12.tif',
#         'df11_11':'../../_data/rasters_peru/R11_C11.tif',
#         'df11_12':'../../_data/rasters_peru/R11_C12.tif',
#         'df12_11':'../../_data/rasters_peru/R12_C11.tif',
#         'df12_12':'../../_data/rasters_peru/R12_C12.tif'
# }

# # Diccionario para almacenar los DataFrames
# statistics = {}

# # Bucle para procesar cada archivo raster
# for df_name, raster_path in rasters.items():
#     # Calcular estadísticas zonales
#     stats = zonal_stats(gdf, raster_path, stats="count", categorical=True, all_touched=True)
    
#     # Crear un DataFrame para las estadísticas
#     df_stats = pd.DataFrame(stats)

#     # Cruce con shp
#     df = pd.concat([gdf.reset_index(drop=True), df_stats], axis=1)
    
#     # Almacenar el DataFrame en el diccionario
#     statistics[df_name] = df


In [ ]:
# dfs_list = list(statistics.values())

# # Concatenar todos los DataFrames en uno solo
# df_consolidado = pd.concat(dfs_list, ignore_index=True)

# # print(df_consolidado)

### Importar CSV (opcional al loop de los rasters)

In [ ]:
# Lee el archivo CSV y carga los datos en un DataFrame
df_consolidado = pd.read_csv('output_file.csv')

# Muestra las primeras filas del DataFrame
# print(df_consolidado)

In [ ]:
# Modificar la columna de CCDD

def add_leading_zero(num):
    num_str = str(num) 
    if len(num_str) == 1:
        return num_str.zfill(2)  
    return num_str

# Aplicar la función a la columna
df_consolidado['CCDD'] = df_consolidado['CCDD'].apply(add_leading_zero)


### Agrupar para cada departamento

In [ ]:
df_f1 = df_consolidado.iloc[:, 14:31] ##seleccionar columnas para trabajar
df_f2 = df_consolidado.iloc[:, [2]]

df_filtered = pd.concat([df_f1, df_f2], axis=1) ##concatena las columnas filtradas
df_filtered

In [ ]:
df_dep = df_filtered.replace(np.nan, 0 ) \ ##reemplaza NaN por cero
    .groupby('CCDD').sum().reset_index()
df_dep

In [ ]:
dta_final_depa = gdf[['CCDD', 'NOMBDEP','geometry']] \ ##combina df y gdf
    .merge(df_dep, on = ['CCDD'], how = 'left') \
    .groupby(['CCDD', 'NOMBDEP','geometry']).sum().reset_index()

In [ ]:
gdf_depa = gpd.GeoDataFrame(dta_final_depa, geometry='geometry')